## Least Cost Pipeline Construction
This notebook imports the cost distance stack generated by `Create-Cost-Stack.ipynb` and a pipeline raster to compute a least cost "feeder" pipeline configuration. The workflow for this process is:
1. Import the stack of cost distance layers (one for each biogas source): $arr\_stack$.
1. Import the pipeline raster, setting non-pipeline cells to NaN: $arr\_pipeline$.
1. Determine which pixel among the pipeline pixels has the least cost among all farm cost distances rasters. This will serve as the location of the connection point to the existing pipeline: $C_0$.
1. Determine which farm is the source of this minimum point, done by finding which layer (in the stack of cost distance rasters) has the minimum value at that location. This represents the least cost biogas source: $F_0$.
1. Compute the least cost path connecting that farm ($F_0$) to the connection point ($C_0$): $LCP_0$.
1. Update the connection point layer ($C_0$) to include the least cost path ($LCP_0$): $Pipes_0$.
1. Remove the layer associated with $F_0$ from the stack of cost distance rasters ($arr\_CDsk$) and repeat steps 4-7:
    * Locate the minimim value among all remaining cost distance rasters to cells in the Pipes layer ($C_i$)...
    * Idenfity the source farm associated with this minimum ($F_i$)...
    * Compute the least cost path from from $F_i$ to $C_i$...
    * Update the connection point layer ($Pipes_{i+1}$)...

In [1]:
#Import packages
import numpy as np
import pandas as pd
from skimage import graph
from osgeo import gdal
import geopandas as gpd
from shapely.geometry import Point

In [2]:
#Read the stack
arrStack = np.load('../data/DuplinStack.npy')
dims = arrStack.shape
print("There are {0} layers in the stack".format(dims[0]))

There are 464 layers in the stack


In [16]:
#Locate the minimum value and determine which layer it's in
allMin = np.amin(arrStack)
x,y,z = np.where(arrStack == allMin)
print("The min value is in layer {}, at coordinates ({},{})".format(x,y,z))

The min value is in layer [0], at coordinates ([114],[63])


Need to reach back to Creat-Cost-Stacka and create/export a stack of cost rasters for each biogas source

In [18]:
#Compute the LCP from that location to the nearest pipe
mcpConnect = graph.MCP_Connect(arrStack[x])

In [ ]:
mcpConnect.create_connection()

In [13]:
#Remove that layer from the stack
arrStack_0 = np.delete(arrStack,x,axis=0)
arrStack_0.shape

(463, 190, 223)

In [15]:
#Get the next min
allMin = np.amin(arrStack_0)
x,y,z = np.where(arrStack == allMin)
print("The min value ({}) is in layer {}".format(allMin,x))

The min value (2.630235719465958) is in layer [1]
